In [13]:
with open('/Users/annasophia/Desktop/GitHub/CSE185/example-files/hg19chr1.fa', 'r') as file:
    genome = ''
    file.readline()
    for line in file:
        line = line.strip()
        genome = genome + line

In [16]:
# returns all cyclic rotations of text
def CyclicRotations(text):
    rotations = [text]
    
    curr = text[-1] + text[:len(text)-1]
    while curr != text:
        rotations.append(curr)
        curr = curr[-1] + curr[:len(text)-1]
        
    return rotations

In [17]:
# returns the burrows-wheeler transform of text
def BWT(text):
    rotations = CyclicRotations(text)
    sortedRotations = sorted(rotations)
    
    bwt = ''
    for string in sortedRotations:
        bwt = bwt + string[-1]
    
    return bwt

In [19]:
bwt = BWT(genome)

In [20]:
with open('/Users/annasophia/Desktop/GitHub/CSE185/example-files/reads.fq', 'r') as file:
    reads = []
    for line in file:
        line = line.strip()
        reads.append(line)

In [21]:
reads

['@HSQ1004:134:C0D8DACXX:1:1101:1427:17913/1',
 'TAGTT',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:1477:88802/1',
 'AAGGC',
 '+',
 '?@@FB',
 '@HSQ1004:134:C0D8DACXX:1:1101:1529:183767/1',
 'TGACA',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:1657:145276/1',
 'CCCAG',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:1722:190628/1',
 'GGCTA',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:1873:77812/1',
 'AACTC',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:1961:62292/1',
 'CCACA',
 '+',
 '???AD',
 '@HSQ1004:134:C0D8DACXX:1:1101:2060:158708/1',
 'TTAAT',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:2272:50825/1',
 'TCCTT',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:2389:135724/1',
 'TTTAA',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:2657:64131/1',
 'TGCCT',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:2793:148992/1',
 'TTATA',
 '+',
 'CC@FF',
 '@HSQ1004:134:C0D8DACXX:1:1101:2796:135978/1',
 'ATTTG',
 '+',
 'CCCFF',
 '@HSQ1004:134:C0D8DACXX:1:1101:2847:187081/